In [1]:
# import libraries

import boto3, s3fs, awswrangler as wr, pandas as pd

from time import sleep

In [4]:
# set s3 buckets

s3_bucket = 's3://datalake-igti-fabio-edc-desafio-final/staging-zone/'
athena_output = 's3://datalake-igti-fabio-edc-desafio-final/athena-results/'

In [5]:
# create Pandas dataframe

s3 = s3fs.S3FileSystem()

df = wr.s3.read_parquet(s3_bucket)

print(df.shape)
print(df.columns)
print(df.dtypes)
df.head()

(5783109, 76)
Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', '

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,200006271946,2020,11,F,1,2,1,1,11,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,200001195856,2020,11,M,2,3,1,1,11,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,200001943954,2020,4,F,2,3,2,2,0,2,...,B,A,A,B,A,A,A,A,A,A
3,200001908998,2020,2,M,1,3,1,2,0,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,200001634757,2020,4,F,1,3,2,1,1,1,...,A,A,A,B,A,B,B,A,A,B


In [11]:
# create client to AWS Athena

client = boto3.client('athena')

# function to build query

def athena_query(sql, athena_output):

    response = client.start_query_execution(
        QueryString = sql,
        QueryExecutionContext = {'Database': 'edc_desafio_final_database'}, 
        ResultConfiguration = {'OutputLocation': athena_output}
    )

    query_status = []

    while not query_status == 'SUCCEEDED':
        query_status= client.get_query_execution(QueryExecutionId=response["QueryExecutionId"])
        query_status = query_status["QueryExecution"]["Status"]["State"]
        sleep(2)

    return response["QueryExecutionId"]

# function to read query results

def query_results(query_id):

    response = client.get_query_results(
        QueryExecutionId=query_id
    )

    return response

# function to convert query results to dataframe

def results_to_df(query_results):
 
    columns = [
        col['Label']
        for col in query_results['ResultSet']['ResultSetMetadata']['ColumnInfo']
    ]
 
    listed_results = []

    for res in query_results['ResultSet']['Rows'][1:]:         
        values = []

        for field in res['Data']:
            try:
                values.append(list(field.values())[0]) 
            except:
                values.append(list(' '))
 
        listed_results.append(
            dict(zip(columns, values))
        )
 
    return pd.DataFrame(listed_results)

In [14]:
# Pergunta 1: Quantos alunos não quiseram declarar a cor/raça em 2020?

print(df.loc[df['TP_COR_RACA'] == 0].shape[0])

sql = ('''

    SELECT 
        COUNT(*)
        
    FROM staging_zone

    WHERE
        TP_COR_RACA = 0

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

116883


,_col0
0,116883


In [15]:
df['CO_UF_ESC'].unique()

<IntegerArray>
[<NA>,   29,   35,   13,   22,   51,   33,   53,   23,   31,   32,   43,   27,
   42,   41,   24,   28,   25,   11,   26,   16,   15,   52,   21,   50,   17,
   14,   12]
Length: 28, dtype: Int32

In [16]:
# Pergunta 2: Qual é o número de alunos do Sexo Feminino que estudaram em escola no estado de São Paulo?

print(df.loc[((df['TP_SEXO'] == 'F') & (df['CO_UF_ESC'] == 35))].shape[0])

sql = ('''

        SELECT
            COUNT(*)
            
        FROM staging_zone

        WHERE 
            TP_SEXO = 'F' AND
            CO_UF_ESC = 35


    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

91480


,_col0
0,91480


In [18]:
# Pergunta 3: Quantos alunos do sexo feminino que estudaram em escola no estado do Rio Grande do Sul possuem EXATAMENTE uma geladeira em casa?

print(df.loc[((df['TP_SEXO'] == 'F') & (df['CO_UF_ESC'] == 43) & (df['Q012'] == 'B'))].shape[0])

sql = ('''

        SELECT
            COUNT(*)
            
        FROM staging_zone

        WHERE 
            TP_SEXO = 'F' AND
            CO_UF_ESC = 43 AND
            Q012 = 'B'


    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

18680


,_col0
0,18680


In [22]:
# Pergunta 4: Qual é o segundo estado brasileiro em que estudaram mais alunos no ENEM 2020?

print(df.groupby('SG_UF_ESC').size().sort_values(ascending=False).head(5))

sql = ('''

        SELECT
            SG_UF_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        GROUP BY SG_UF_ESC

        ORDER BY COUNT(*) DESC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

SG_UF_ESC
SP    167183
CE    104981
MG     71614
RJ     62583
PE     47583
dtype: int64


,SG_UF_ESC,QTD
0,[ ],4878540
1,SP,167183
2,CE,104981
3,MG,71614
4,RJ,62583


In [23]:
# Pergunta 5: Qual é o estado brasileiro (considere a coluna SG_UF_ESC) possui o menor número de alunos cuja mãe possui ensino superior completo?

print(df.loc[df['Q002'] == 'F'].groupby('SG_UF_ESC').size().sort_values().head(5))

sql = ('''

        SELECT
            SG_UF_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            Q002 = 'F'

        GROUP BY SG_UF_ESC

        ORDER BY COUNT(*) ASC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

SG_UF_ESC
RR     425
AC     491
AP     494
SE     936
RO    1002
dtype: int64


,SG_UF_ESC,QTD
0,RR,425
1,AC,491
2,AP,494
3,SE,936
4,RO,1002


In [25]:
# Pergunta 6: Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de pessoas na faixa “entre 26 e 30 anos”?

print(df.loc[df['TP_FAIXA_ETARIA'] == 11].groupby('SG_UF_ESC').size().sort_values(ascending=False).head(5))

sql = ('''

        SELECT
            SG_UF_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            TP_FAIXA_ETARIA = 11

        GROUP BY SG_UF_ESC

        ORDER BY COUNT(*) DESC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

SG_UF_ESC
CE    1880
AM     828
SP     580
GO     518
PA     479
dtype: int64


,SG_UF_ESC,QTD
0,[ ],567071
1,CE,1880
2,AM,828
3,SP,580
4,GO,518


In [26]:
# Pergunta 7: Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o TERCEIRO maior número de alunos cuja residência possui PELO MENOS 2 banheiros?

print(df.loc[((df['Q008'] == 'C') | (df['Q008'] == 'D') | (df['Q008'] == 'E'))].groupby('SG_UF_ESC').size().sort_values(ascending=False).head(5))

sql = ('''

        SELECT
            SG_UF_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            (Q008 = 'C' OR Q008 = 'D' OR Q008 = 'E')

        GROUP BY SG_UF_ESC

        ORDER BY COUNT(*) DESC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

SG_UF_ESC
SP    83502
MG    30912
RJ    24367
PR    20938
CE    15533
dtype: int64


,SG_UF_ESC,QTD
0,[ ],1151573
1,SP,83502
2,MG,30912
3,RJ,24367
4,PR,20938


In [27]:
# Pergunta 8: Quantos alunos do sexo feminino se autodeclararam pretos?

print(df.loc[((df['TP_SEXO'] == 'F') & (df['TP_COR_RACA'] == 2))].shape[0])

sql = ('''

        SELECT
            COUNT(*)
            
        FROM staging_zone

        WHERE 
            TP_SEXO = 'F' AND
            TP_COR_RACA = 2


    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

444346


,_col0
0,444346


In [28]:
# Pergunta 9: Quantos alunos estrangeiros fizeram o ENEM 2020?

print(df.loc[df['TP_NACIONALIDADE'] == 3].shape[0])

sql = ('''

        SELECT
            COUNT(*)
            
        FROM staging_zone

        WHERE 
            TP_NACIONALIDADE = 3


    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

8036


,_col0
0,8036


In [31]:
# Pergunta 10: Qual é a diferença da nota média em matemática dos alunos que estudaram o ensino médio em escola pública e em escola privada?

print(df[['TP_ESCOLA', 'NU_NOTA_MT']].groupby('TP_ESCOLA').agg('NU_NOTA_MT').mean().sort_values(ascending=False))

sql = ('''

        SELECT
            TP_ESCOLA,
            AVG(NU_NOTA_MT)
            
        FROM staging_zone

        WHERE 
            (TP_ESCOLA = 2 OR TP_ESCOLA = 3)

        GROUP BY TP_ESCOLA

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

TP_ESCOLA
3    610.639357
1    520.033425
2    499.524980
Name: NU_NOTA_MT, dtype: float64


,TP_ESCOLA,_col1
0,3,610.6393569797273
1,2,499.5249803844639


In [33]:
# Pergunta 11: Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de alunos do sexo feminino indígenas?

print(df.loc[((df['TP_SEXO'] == 'F') & (df['TP_COR_RACA'] == 5))].groupby('SG_UF_ESC').size().sort_values(ascending=False).head(5))

sql = ('''

        SELECT
            SG_UF_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            TP_SEXO = 'F' AND
            TP_COR_RACA = 5

        GROUP BY SG_UF_ESC

        ORDER BY COUNT(*) DESC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

SG_UF_ESC
AM    398
CE    380
SP    298
PE    249
BA    200
dtype: int64


,SG_UF_ESC,QTD
0,[ ],18081
1,AM,398
2,CE,380
3,SP,298
4,PE,249


In [45]:
# Pergunta 12: Qual é a diferença entre o número de alunos cujo pai possui pós-graduação completa e o número de alunos cuja mãe possui pós-graduação completa?

print(df.loc[((df['Q001'] == 'G') & ~(df['Q002'] == 'G'))].shape[0] - df.loc[(~(df['Q001'] == 'G') & (df['Q002'] == 'G'))].shape[0])

sql = ('''

        SELECT
            SUM(CASE
                WHEN Q001 = 'G' AND NOT Q002 = 'G' THEN 1
                ELSE 0
                END) AS PAI_POS,
            SUM(CASE
                WHEN NOT Q001 = 'G' AND Q002 = 'G' THEN 1
                ELSE 0
                END) AS MAE_POS

        FROM staging_zone

        LIMIT 10

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

-194717


,PAI_POS,MAE_POS
0,125238,319955


In [67]:
# Pergunta 13: Quantos alunos cuja residência possui ATÉ 2 carros estudaram na região NORDESTE do Brasil?

print(df.loc[((df['CO_MUNICIPIO_ESC'].astype(str).str[0] == '2') & ((df['Q010'] == 'B') | (df['Q010'] == 'C')))].shape[0])

sql = ('''

        SELECT
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            CAST(CO_MUNICIPIO_ESC AS VARCHAR) LIKE '2%' AND
            (Q010 = 'B' OR Q010 = 'C')

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

74533


,QTD
0,74533


In [63]:
# Pergunta 14: Quantos alunos que estudaram em escolas em zona rural possuem internet em casa?

print(df.loc[((df['TP_LOCALIZACAO_ESC'] == 2) & (df['Q025'] == 'B'))].shape[0])

sql = ('''

        SELECT
            COUNT(*) AS QTD
            
        FROM staging_zone

        WHERE
            TP_LOCALIZACAO_ESC = 2 AND
            Q025 = 'B'

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

18979


,QTD
0,18979


In [64]:
# Pergunta 15: Qual é o nome do município que contém a SEGUNDA maior quantidade total de inscritos no ENEM 2020?

print(df.groupby('NO_MUNICIPIO_ESC').size().sort_values(ascending=False).head(5))

sql = ('''

        SELECT
            NO_MUNICIPIO_ESC,
            COUNT(*) AS QTD
            
        FROM staging_zone

        GROUP BY NO_MUNICIPIO_ESC

        ORDER BY COUNT(*) DESC

        LIMIT 5

    ''')

results_to_df(query_results(athena_query(sql=sql, athena_output=athena_output)))

NO_MUNICIPIO_ESC
S�o Paulo         38277
Fortaleza         27783
Rio de Janeiro    26431
Bras�lia          19840
Manaus            17120
dtype: int64


,NO_MUNICIPIO_ESC,QTD
0,[ ],4878540
1,S�o Paulo,38277
2,Fortaleza,27783
3,Rio de Janeiro,26431
4,Bras�lia,19840
